# import def func

In [1]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [2]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
#     tdx_df = f'lastpTDX_DF_{resample}'
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [3]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [4]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [5]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-20',
 code
 300763    锦浪科技
 Name: name, dtype: object)

In [6]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/22/25 21:59:00'

code
839493   2025-06-22 15:30:00
603119   2025-06-22 15:00:03
Name: ticktime, dtype: datetime64[ns]

418.96660226980845

#### 实时成交

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:G:\sina_MultiIndex_data.h5
[]
G:\sina_MultiIndex_data.h5


#### 读取实时成交个股

In [8]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

##### 合并实时成交量及SinaDM

In [9]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [10]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [11]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [12]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [13]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5184 it:0.5 lastpTDX_DF_3d not in locals
keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-06-22 21:59:05,686] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:3 code:['601028', '600837']


tdxdataT:3.21
DLTDXE:3.21 

In [39]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:1214 216


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,29.98,0,2025-04-02,52.53,43.47,13.18,50.31,15.33,0.0,63.19
301038,深水规院,20.09,23,2025-04-08,17.27,14.98,11.53,57.31,11.53,0.1,0.00
301292,海科新源,20.06,60,2025-04-08,23.52,15.50,10.95,63.59,11.46,0.0,1.30
300480,光力科技,20.02,85,2025-04-08,15.59,14.28,10.36,40.31,10.98,0.0,0.20
300589,江龙船艇,20.00,114,2025-04-08,15.12,13.02,10.10,60.41,10.10,0.1,-0.10
300340,科恒股份,20.00,236,2025-04-08,20.64,18.86,7.66,35.31,7.90,0.0,64.31
300410,正业科技,19.97,83,2025-04-08,7.45,6.12,4.00,46.41,4.12,0.0,1.30
300485,赛升药业,19.97,138,2025-04-17,13.94,13.69,6.95,1.10,7.38,0.1,42.59
301150,中一科技,18.56,26,2025-04-08,21.91,17.03,12.01,58.09,12.35,0.0,7.30


In [15]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

lastdu4       1.07
boll             1
ral             10
fib              1
percmax        4.0
macd          0.59
macddif       0.26
macddea      -0.04
macdlast2     0.28
macdlast3     0.37
lasth1d      17.27
lastl1d       14.4
lastl2d      13.92
lastl3d      14.67
lastp1d      17.27
ma51d        15.13
ma52d        14.59
ma53d        14.61
ma54d         14.6
Name: 301038, dtype: object

In [16]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [17]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5184 it:0.7 lastpTDX_DF_w not in locals
keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [45]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:288 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,29.98,0,2025-01-27,52.53,30.20,11.71,74.69,11.91,1.2,1.20
301038,深水规院,20.09,23,2025-01-03,17.27,17.41,11.53,41.59,12.98,0.1,1.20
301292,海科新源,20.06,60,2025-01-10,23.52,15.37,10.95,80.19,11.83,0.1,1.20
300429,强力新材,20.02,259,2025-01-27,14.27,14.76,9.70,48.91,10.63,0.1,0.00
300410,正业科技,19.97,83,2025-04-18,7.45,7.09,4.00,43.91,4.67,27.3,0.00
301150,中一科技,18.56,26,2025-04-11,21.91,18.88,12.01,52.91,12.01,1.3,0.00
839719,宁新新材,15.72,0,2025-01-27,23.70,15.48,8.75,77.38,8.81,0.0,40.59
300269,联建光电,13.21,532,2025-04-11,6.17,5.41,2.80,71.88,2.80,20.3,-0.10
300950,德固特,12.71,6,2025-01-03,21.20,18.02,12.74,44.41,14.62,0.1,1.30


In [19]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

### resample M

In [20]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5184 it:0.7 lastpTDX_DF_m not in locals
keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [41]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:1876 216


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,29.98,0,2024-09-30,52.53,20.06,6.20,135.25,6.65,0.0,39.00
301038,深水规院,20.09,23,2025-04-30,17.27,16.12,9.53,1.30,11.53,-1.0,0.20
301292,海科新源,20.06,60,2025-01-27,23.52,16.92,10.86,73.19,11.73,0.0,30.80
300429,强力新材,20.02,259,2025-01-27,14.27,14.86,9.02,1.30,10.50,-1.1,37.59
300340,科恒股份,20.00,236,2025-04-30,20.64,11.83,6.26,123.50,7.66,-1.1,31.91
300589,江龙船艇,20.00,114,2025-04-30,15.12,13.57,10.03,1.30,10.10,-1.1,0.10
300485,赛升药业,19.97,138,2025-01-27,13.94,8.10,5.73,79.62,6.31,0.1,1.20
300410,正业科技,19.97,83,2025-01-27,7.45,6.71,4.30,1.30,4.67,0.0,23.30
301150,中一科技,18.56,26,2025-04-30,21.91,18.88,12.45,52.00,12.01,-1.1,34.09


In [22]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

lasth1d    17.27
lastl1d    13.92
lastl2d    13.39
lastl3d    11.53
lastp1d    17.27
ma51d       14.7
ma52d      13.97
ma53d      13.92
ma54d      14.48
Name: 301038, dtype: object

### resample D

In [23]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)
h5_d.shape

initdx :0 b1>:5184 it:0.6 lastpTDX_DF_d not in locals
keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-06-22 21:59:39,311] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:3 code:['601028', '600837']


tdxdataT:3.67
DLTDXE:3.67 

(5180, 218)

In [42]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lasth1d > lasth2d  and ral > 10 and fib > 0 and boll > 0  and percent > lastdu4 \
and close >high4  and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
if len(td_code) < 30:
    write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:63 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,29.98,0,2025-04-07,52.53,43.47,18.44,52.31,13.18,0.2,0.0
301038,深水规院,20.09,23,2025-04-07,17.27,15.36,13.51,40.31,11.53,0.0,-0.1
301292,海科新源,20.06,60,2025-04-07,23.52,16.33,14.52,64.50,11.46,0.1,0.0
300429,强力新材,20.02,259,2025-04-28,14.27,12.72,10.92,40.31,10.44,0.0,0.1
300485,赛升药业,19.97,138,2025-04-07,13.94,13.89,7.99,28.30,7.03,0.0,0.1
300410,正业科技,19.97,83,2025-04-07,7.45,6.21,5.58,45.91,4.12,0.0,0.0
301150,中一科技,18.56,26,2025-04-07,21.91,17.03,15.16,55.41,12.35,0.1,0.0
839719,宁新新材,15.72,0,2025-04-07,23.70,18.86,11.26,41.81,8.90,1.1,0.0
603616,韩建河山,10.12,27,2025-04-08,4.46,4.27,3.72,40.41,2.94,0.0,0.1


In [25]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

lastdu4       1.05
close        17.27
boll             1
ral             15
fib              2
percmax        5.0
macd          0.24
macddif       0.34
macddea       0.22
macdlast1     0.24
macdlast2    -0.12
macdlast3    -0.09
lasth1d      17.27
lastl1d       14.4
lastl2d      14.22
lastl3d      13.92
lastp1d      17.27
ma51d        15.03
ma52d        14.57
ma53d        14.76
ma54d        14.82
Name: 301038, dtype: object

#### filter test

In [34]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=5)

Count:219 216


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300804,广康生化,7.87,1,2025-04-11,60.88,57.91,20.86,1.10,21.44,1.10,1.2
688313,仕佳光子,6.93,57,2025-04-08,38.59,35.91,12.74,33.19,13.55,1.20,1.2
300204,舒泰神,5.45,18,2025-04-08,35.00,36.94,5.90,1.10,6.05,20.91,0.0
688185,康希诺,5.13,0,2025-04-08,60.81,62.03,48.50,1.10,48.50,0.00,27.3
688379,华光新材,4.96,0,2025-04-08,31.95,33.75,17.38,1.10,18.78,0.00,0.0


In [35]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [46]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
if len(code_f) > 0:
    show_tdx_data(td_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:1876
tw_code:288
t3d_code:1214
td_code:63
Count:20 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,29.98,0,2025-04-07,52.53,43.47,18.44,52.31,13.18,0.2,0.0
301038,深水规院,20.09,23,2025-04-07,17.27,15.36,13.51,40.31,11.53,0.0,-0.1
301292,海科新源,20.06,60,2025-04-07,23.52,16.33,14.52,64.50,11.46,0.1,0.0
300410,正业科技,19.97,83,2025-04-07,7.45,6.21,5.58,45.91,4.12,0.0,0.0
301150,中一科技,18.56,26,2025-04-07,21.91,17.03,15.16,55.41,12.35,0.1,0.0
839719,宁新新材,15.72,0,2025-04-07,23.70,18.86,11.26,41.81,8.90,1.1,0.0
603616,韩建河山,10.12,27,2025-04-08,4.46,4.27,3.72,40.41,2.94,0.0,0.1
002235,安妮股份,9.99,463,2025-04-08,9.03,7.33,5.17,38.00,4.33,22.3,10.5
603948,建业股份,9.98,5,2025-04-08,19.61,19.02,17.66,0.10,14.81,-0.1,-0.1


#### check data

In [37]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

3

In [30]:
show_tdx_data(td_code.loc[code_f])

Count:3 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300816,艾可蓝,3.07,1,2025-04-07,35.62,34.66,29.45,1.2,24.34,1.3,1.1
600926,杭州银行,2.75,142,2025-04-07,17.16,16.88,14.86,1.3,13.58,0.0,0.0
601555,东吴证券,2.57,229,2025-04-07,8.77,8.76,7.57,1.3,6.77,-1.1,31.8


### Write toblk

In [31]:
# write_code_to_blk(code_f,'077',rewrite=False)